# Lesson Seven

In [2]:
# Let's import from lesson_six
import lesson_six as lesson_six
# Rest the graph
lesson_six.reset_graph()

In [3]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

# Instanciat the data
housing = fetch_california_housing()

# The shape of the data is (20640, 8) this will return m is the number of rows, n is the number of column
m, n = housing.data.shape 
#housing_data_and_bias = np.c_[np.ones((m, 1)), housing.data]

# We have to standardiaze the input data
from sklearn.preprocessing import StandardScaler

# Instantiate the preprocessing object
scaler = StandardScaler() 

# This will standardize the input data, the shape will be (m, 8)
scaled_housing_data = scaler.fit_transform(housing.data) 

# This will add bias, this will add column at index 0, the shape will be (m, 8+1) 
scaled_housing_data_and_bias = np.c_[np.ones((m, 1)), scaled_housing_data] 

# Using TensorFlow's Gradient Descent


In [4]:
lesson_six.reset_graph()

n_epochs = 1000
learning_rate = 0.01

# Wrap the scaled_housing_data_and_bias with tf.constant()
X = tf.constant(scaled_housing_data_and_bias, dtype=tf.float32, name="X")

# Wrpa the housing.target (reshape it from (n,) to (n,1)) with tf.constant()
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# Compute the theta with tf.random_uniform() use the number of column add 1 so it will match the X+bias columns
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=1), name="theta")

# Compute the y_hat using tf.matmul()
y_pred = tf.matmul(X, theta, name="predictions")

# Get the error
error = y_pred - y

# Compute the Mean Square Error using tf.reduce_mean() and tf.square(error)
mse = tf.reduce_mean(tf.square(error), name="mse")

In [5]:
# TensorFlow's autodiff feature can automatically compute the gradient for us. 
# Simply add the following line of code.
gradients = tf.gradients(mse, [theta])[0]

The tf.gradients() function takes mse and a list  of variables [theta] and it creates a list of operations one per variable to compute the gradients of the operation with regards to each variable. So this will return the gradient vector of the MSE with regards to theta.

In [6]:
# Optimizer
training_op = tf.assign(theta, theta - learning_rate * gradients)

# Globa variable initializer
global_init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(global_init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE= ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    
print("Best theta: ", best_theta)

Epoch:  0 MSE=  14.5535
Epoch:  100 MSE=  0.837282
Epoch:  200 MSE=  0.561952
Epoch:  300 MSE=  0.546907
Epoch:  400 MSE=  0.540841
Epoch:  500 MSE=  0.536597
Epoch:  600 MSE=  0.533485
Epoch:  700 MSE=  0.531191
Epoch:  800 MSE=  0.529492
Epoch:  900 MSE=  0.528231
Best theta:  [[ 2.06855226]
 [ 0.84201318]
 [ 0.13798265]
 [-0.25731888]
 [ 0.28500068]
 [ 0.00233129]
 [-0.04121713]
 [-0.7480495 ]
 [-0.71861744]]


In [7]:
lesson_six.reset_graph()

n_epochs = 1000
learning_rate = 0.01

# Wrap the scaled_housing_data_and_bias with tf.constant()
X = tf.constant(scaled_housing_data_and_bias, dtype=tf.float32, name="X")

# Wrpa the housing.target (reshape it from (n,) to (n,1)) with tf.constant()
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# Compute the theta with tf.random_uniform() use the number of column add 1 so it will match the X+bias columns
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

# Compute the y_hat using tf.matmul()
y_pred = tf.matmul(X, theta, name="predictions")

# Get the error
error = y_pred - y

# Compute the Mean Square Error using tf.reduce_mean() and tf.square(error)
mse = tf.reduce_mean(tf.square(error), name="mse")

TensorFlow also provides a number of optimizer, such as the GradientDescentOptimizer() and the MeMentumOptimizer()

In [8]:
# Gradient Decscent with optimizer
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5)
training_op = optimizer.minimize(mse)

In [9]:
# Globa variable initializer
global_init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    # Pass the global_init to the sess.run()
    sess.run(global_init)
    
    # Iterate for training 
    for epoch in range(n_epochs):
        
        # Print the MSE every 100 iteration
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE= ", mse.eval())
            
        # Pass the optimizer.minimize(mse) on each iteration
        sess.run(training_op)
        
    # Evaluate the theta to get a list of the best theta (9,1)
    best_theta = theta.eval()
    
print("Best theta: ", best_theta)

Epoch:  0 MSE=  12.2318
Epoch:  100 MSE=  0.798976
Epoch:  200 MSE=  0.66714
Epoch:  300 MSE=  0.599869
Epoch:  400 MSE=  0.564523
Epoch:  500 MSE=  0.545875
Epoch:  600 MSE=  0.535987
Epoch:  700 MSE=  0.530709
Epoch:  800 MSE=  0.527867
Epoch:  900 MSE=  0.526322
Best theta:  [[  2.06855559e+00]
 [  8.37727904e-01]
 [  1.30713731e-01]
 [ -2.61316836e-01]
 [  2.93626070e-01]
 [ -2.56629748e-04]
 [ -4.05110605e-02]
 [ -8.05071473e-01]
 [ -7.75725722e-01]]


In [10]:
lesson_six.reset_graph()

## Placeholder Nodes

Placeholder nodes is just to output the data at runtime, it dosen't perform any computation, and it is used to pass the training data to TensorFlow during trainin.

In [11]:
# Create a placeholder node, we invoke the placeholder() function, and pass the data type and the shape(optional)
# Here we define the spape to be (n, 3) any rows, but 3 columns
A = tf.placeholder(tf.float32, shape=(None, 3))

# B is a node depends on A
B = A + 5

# Now lets do the computation
with tf.Session() as sess:
    # Evaluate B, by passing data to A using feed_dict={...}
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})



In [12]:
print(B_val_1)

[[ 6.  7.  8.]]


In [13]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [14]:
lesson_six.reset_graph()

# Mini-batch Gradient Descent

To implement Mini-batch Gradient Descent we will use a placeholder() node.

In [15]:
n_epochs = 1000
learning_rate = 0.01

In [16]:
# Change the definition of X and y in the construction phase to make them a placeholder nodes.
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# Compute the theta with tf.random_uniform() use the number of column add 1 so it will match the X+bias columns
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

# Compute the y_hat using tf.matmul()
y_pred = tf.matmul(X, theta, name="predictions")

# Get the error
error = y_pred - y

# Compute the Mean Square Error using tf.reduce_mean() and tf.square(error)
mse = tf.reduce_mean(tf.square(error), name="mse")

# Gradient Decscent with optimizer
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5)
training_op = optimizer.minimize(mse)

In [17]:
# Define the batch size and compute the total number of batches:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [18]:
# In the execution phase: we will fetch the mini-batches one by one, then provide the value of X and y via using 
# feed_dict parameter when evaluating a node that depends on either of them

def fetch_batch(epoch, batch_index, batch_size):
    '''
    This fucntion will return the X_batch, y_batch with random indices
    '''
    
    # Find a random indices whithin the size of the dataset
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    
    # Pich the X and y batches using the random indices
    X_batch = scaled_housing_data_and_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [18]:
# Globa variable initializer
global_init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(global_init)

    for epoch in range(n_epochs):
        
        # Print the MSE every 100 iteration
        if epoch % 100 == 0:
            print("Epoch: ", epoch)
        
        for batch_index in range(n_batches):
            
            # Get the X_batch and the y_batch from the function fetch_batch()
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            # Evaluate the 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    # We don't need to pass the value X and y when evaluating theta since it does not depend on either of them.
    best_theta = theta.eval()

Epoch:  0
Epoch:  100
Epoch:  200
Epoch:  300
Epoch:  400
Epoch:  500
Epoch:  600
Epoch:  700
Epoch:  800
Epoch:  900


In [19]:
best_theta

array([[ 2.049927  ],
       [ 0.8267504 ],
       [ 0.11433333],
       [-0.23890038],
       [ 0.31248516],
       [ 0.03146332],
       [-1.4988426 ],
       [-0.87141562],
       [-0.83012849]], dtype=float32)

# Saving and restoring a model
TensorFlow makes saving and restoring a model very easy. We only need to create a Server node after all variables nodes are created, then, during running phase just invoke the save() method, passing it the session and the path of the file:

In [23]:
lesson_six.reset_graph()

n_epochs = 1000                                                     
learning_rate = 0.01                                                                

# Change the definition of X and y in the construction phase to make them a placeholder nodes.
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# Compute the theta with tf.random_uniform() use the number of column add 1 so it will match the X+bias columns
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

# Compute the y_hat using tf.matmul()
y_pred = tf.matmul(X, theta, name="predictions")   

# Get the error
error = y_pred - y             

# Compute the Mean Square Error using tf.reduce_mean() and tf.square(error)
mse = tf.reduce_mean(tf.square(error), name="mse")        

# Gradient Decscent with optimizer
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer =  tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.5) 
training_op = optimizer.minimize(mse) 

# Define the batch size and compute the total number of batches:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# Globa variable initializer
global_init = tf.global_variables_initializer()

# Invoke saver() here at the end of construction phase
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(global_init)

    for epoch in range(n_epochs):
        
        # Print the MSE every 100 iteration
        if epoch % 100 == 0:
            print("Epoch: ", epoch)
            
            # Save
            save_path = saver.save(sess, "/Volumes/MacAndroidStudio/ml-books/savedModel/my_model.ckpt")
        
        for batch_index in range(n_batches):
            
            # Get the X_batch and the y_batch from the function fetch_batch()
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            # Evaluate the 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    # We don't need to pass the value X and y when evaluating theta since it does not depend on either of them.
    best_theta = theta.eval()
    
    # Save
    save_path = saver.save(sess, "/Volumes/MacAndroidStudio/ml-books/savedModel/my_model_final.ckpt")

Epoch:  0
Epoch:  100
Epoch:  200
Epoch:  300
Epoch:  400
Epoch:  500
Epoch:  600
Epoch:  700
Epoch:  800
Epoch:  900


In [24]:
best_theta

array([[ 2.049927  ],
       [ 0.8267504 ],
       [ 0.11433333],
       [-0.23890038],
       [ 0.31248516],
       [ 0.03146332],
       [-1.4988426 ],
       [-0.87141562],
       [-0.83012849]], dtype=float32)

To restore, we can create a tf.train.Saver() just to replace tf.global_variables_initializer() and then within the with block we invoke saver.restore() pass the sess instance and the path as argument.


In [25]:
# This will restore a saved model, including both the graph structure and the variable values, 
# without having to search for the code that built it.
with tf.Session() as sess:
    saver.restore(sess, "/Volumes/MacAndroidStudio/ml-books/savedModel/my_model_final.ckpt")
    best_theta_restored = theta.eval()

In [26]:
best_theta_restored

array([[ 2.049927  ],
       [ 0.8267504 ],
       [ 0.11433333],
       [-0.23890038],
       [ 0.31248516],
       [ 0.03146332],
       [-1.4988426 ],
       [-0.87141562],
       [-0.83012849]], dtype=float32)